# SUB-BENCHMARK 1: Comparing jDRs performances on simulated data

We here reproduce the first sub-benchmark present in the paper. The performances of the 9 multi-omics jDRs are thus compared based on their clustering performances on simulated datasets.

## Construction of the simulated datasets

We first define the function that produces the simulated datasets. The function simulates 3-omics datasets (mRNA expression, methylation and protein quantification), using the CRAN InserSIM package. Each dataset is composed of 100 samples. We considered different scenarios. First, 'equally spaced' clusters (i.e., composed of the same number of samples) and heterogeneous clusters (i.e., composed of a variable number of samples) can be generated. Second, different numbers of clusters are imposed on the data: 5,10,15. 

In [1]:
library("InterSIM", quietly = TRUE)
source("runfactorization.R")


Attaching package: ‘pkgmaker’

The following object is masked from ‘package:base’:

    isFALSE

NMF - BioConductor layer [OK] | Shared memory capabilities [NO: bigmemory] | Cores 7/8
  To enable shared memory capabilities, try: install.extras('
NMF
')
Loading required package: mclust
Package 'mclust' version 5.4.5
Type 'citation("mclust")' for citing this R package in publications.
Loading required package: ade4

Attaching package: ‘ade4’

The following object is masked from ‘package:BiocGenerics’:

    score


Attaching package: ‘GPArotation’

The following object is masked from ‘package:NMF’:

    entropy


Attaching package: ‘MOFAtools’

The following objects are masked from ‘package:NMF’:

    featureNames, featureNames<-, predict, sampleNames, sampleNames<-

The following objects are masked from ‘package:Biobase’:

    featureNames, featureNames<-, sampleNames, sampleNames<-

The following object is masked from ‘package:stats’:

    predict

Loading required package: JADE
Loadin

In [2]:
# Base folder for data
data_folder <- "../data/"
# Label to identify current run
tag <- format(Sys.time(), "%Y%m%d%H%M%S")
# Folder containing simulated data
simul_folder <- paste0(data_folder, "simulations_", tag, "/") 
# Folder for comparison results
results_folder <- paste0("../results", tag, "/")

dir.create(data_folder, showWarnings = FALSE)
dir.create(simul_folder, showWarnings = FALSE)
dir.create(results_folder, showWarnings = FALSE)

In [3]:
## Simulate data
## INPUTS:
# folder = location  where the simulated data should be saved
# num.clusters = number of clusters to be imposed on the data
# size = heterogeneous for heterogeneous clusters, equal for equally-sized clusters
## OUPUTS: matrices of simulated data are saved to file in folder
simulated_data_generation <- function(out.folder, num.clusters, size="heterogeneous", predefined=TRUE) {
    
    # Number of clusters
    num.clusters <- as.numeric(num.clusters)
    # Size of the effect
    effect <- 2.5
    # Sample proportions per clusters defined here are those used for the paper
    prop_predefined <- list(
        "heterogeneous" = list(
            "5" = c(0.35, 0.13, 0.19, 0.08, 0.25),
            "10" = c(0.20, 0.10, 0.07, 0.10, 0.15, 0.13, 0.10, 0.08, 0.05, 0.02),
            "15" = c(0.10,0.08,0.04,0.03,0.12,0.03,0.10,0.03,0.05,0.02,0.1,0.2,0.03,0.02,0.05)
        ),
        "equal" = list(
            "5" = c(0.25,0.2,0.2,0.2,0.15),
            "10" = c(0.15,0.1,0.1,0.1,0.1,0.1,0.05,0.1,0.1,0.1),
            "15" = c(0.07,0.07,0.07,0.06,0.07,0.07,0.07,0.06,0.07,0.06,0.07,0.06,0.07,0.06,0.07)
        )
    )

    # Check provided parameter (size) against allowed values
    if(! size %in% names(prop_predefined)) {
        print(paste0("ERROR: size can only assume value : ", 
                     paste0(names(prop_predefined), collapse=","),
                     " found : ", size))
    }

    # If article proportions are to be used
    if(predefined) {
        # Check provided parameter (number of clusters) against allowed values
        if(! as.character(num.clusters) %in% names(prop_predefined[[size]])) {
            print(paste0("ERROR: num.clusters can only assume value : ", 
                         paste0(names(prop_predefined[[size]]), collapse=","),
                         " found : ",
                         num.clusters))
        }
        prop <- prop_predefined[[size]][[as.character(num.clusters)]]
        prop[1] <- 1-sum(prop[-1])
    }
    # Otherwise
    else {
        if(size == "equal") {
            # Could be simplified! Only necessary because InterSIM is "easily offended" :
            # ensure same object type as in the heterogeneous case, and that not all 
            # values are exactly the same (should not impact the number of samples per group)
            # - same type
            equals <- rep(1, num.clusters)
            prop <- equals/sum(equals)
            # - slightly imbalanced
            delta <- 0.05*prop[1]
            prop[1] <- prop[1]+delta
            prop[num.clusters] <- prop[num.clusters]-delta
            # - sum is 1
            prop <- round(prop, digits = 10)
            prop[1] <- 1-sum(prop[-1])
        }
        else {
            random <- runif(n = num.clusters, min = 0, max = 1)
            prop <- random/sum(random)
        }
    }

    # Simulate data based on provided parameters
    print(prop)
    print(sum(prop))
    print(sum(prop)==1)
    sim.D <- InterSIM(n.sample=100, cluster.sample.prop=prop, 
                      delta.methyl=effect, delta.expr=effect, delta.protein=effect, 
                      p.DMP=0.25, p.DEG=NULL, p.DEP=NULL,
                      do.plot=FALSE, sample.cluster=TRUE, feature.cluster=TRUE)

    # Export simulations as tables
    write.table(sim.D$clustering.assignment, paste(out.folder, "clusters.txt", sep="/"), sep="\t")
    write_table_with_index_header(t(sim.D$dat.methyl), paste(out.folder, "omics1.txt", sep="/"))
    write_table_with_index_header(t(sim.D$dat.expr), paste(out.folder, "omics2.txt", sep="/"))
    write_table_with_index_header(t(sim.D$dat.protein), paste(out.folder, "omics3.txt", sep="/"))

    return("data saved in folder")
}

## Support function
write_table_with_index_header <- function(data, file, sep="\t") {
    write.table(cbind(probe=row.names(data),data), file, sep = sep, 
                append = FALSE, quote = FALSE, row.names = FALSE, col.names = TRUE)
}

## Comparison based on clustering assignment

We here define the function to compare the clustering performances of the 9 jDRs. The function computes the Jaccard Index of the clusters predicted from the different jDR vs. the ground-truth clusters imposed during simulations. It is to note that, when necessary, the cluster assignment is computed from the factors by kmeans clustering (see Methods of the paper for further details).

In [4]:
## Compare factorizations obtained through different methods
## INPUTS: 
# factorizations = list of factorization results
# methods = list of ran factorization methods
# in.folder = path to input folder
# out.folder = path to output folder
# icluster.clusters = clustering result of iCluster
# intNMF.clusters = clustering result of intNMF
# number_cl = number of clusters
## OUPUTS: matrix of Jaccard Indexes is saved to table in out.folder
clusters_comparison <- function(factorizations, methods, in.folder, out.folder,
                                icluster.clusters, intNMF.clusters, number_cl) {

    ind <- 0
    JI_final <- matrix(data=NA, nrow=number_cl, ncol=length(factorizations))

    # Read clusters imposed on simulated data 
    cl  <- as.matrix(read.table(paste0(in.folder, "clusters.txt"), 
                                sep="\t", row.names=1, header=TRUE))
    cl2 <- as.matrix(as.numeric(cl[,2]))
    rownames(cl2) <- cl[,1]

    # Assigning samples to cluster based on factors
    for (i in 1:length(factorizations)) {
        
        if (methods[i]=="iCluster" || methods[i]=="intNMF") {
            
            if(methods[i]=="iCluster") {
                clust_iCluster <- as.matrix(icluster.clusters)
            } else {
                clust_iCluster <- as.matrix(intNMF.clusters)
            }
            
            # Creation sets of samples
            JI_mat <- numeric(0)
            for (p in 1:number_cl) {
                x1 <- rownames(clust_iCluster)[which(clust_iCluster[,1]==p)]
                row <- numeric(0)
                for (j in 1:number_cl) {
                    x2  <- rownames(cl2)[which(cl2[,1]==j)]
                    I   <- length(intersect(x1,x2))
                    S   <- I/(length(x1)+length(x2)-I)
                    row <- cbind(row,S)
                }
                JI_mat <- rbind(JI_mat,row)
            }
            JI_final[1:number_cl,i]<-apply(JI_mat,1,max)
            
        } else {
            
            factors <- factorizations[[i]][[1]]
            
            # Clustering by Kmeans
            JI_good <- numeric(0)
            for (run in 1:1000) {
                kmeans.out <- kmeans(factors, centers=number_cl)
                kmeans.out
                clust_iCluster <- as.matrix(kmeans.out$cluster)
                ######creation sets of samples
                JI_mat <- numeric(0)
                for (p in 1:number_cl) {
                    x1  <- rownames(clust_iCluster)[which(clust_iCluster[,1]==p)]
                    row <- numeric(0)
                    for(j in 1:number_cl) {
                        x2  <- rownames(cl2)[which(cl2[,1]==j)]
                        I   <- length(intersect(x1,x2))
                        S   <- I/(length(x1)+length(x2)-I)
                        row <- cbind(row,S)
                     }
                     JI_mat <- rbind(JI_mat,row)
                 }
                 JI_good <- rbind(JI_good,apply(JI_mat,1,max))
            }
            JI_final[1:number_cl,i] <- apply(JI_good,2,mean)
        }
    }
    
    # Export JI values to a file 
    write.table(JI_final, paste0(out.folder,"/JI.txt"), sep="\t", row.names=FALSE, col.names=methods)
}

## Running the comparisons

We here run the jDR comparison. First the function to simulate the data, then the factorization through the external function run_factorization and then the function comparing the performances based on the Jaccard Index.

In [5]:
## Simulate data, factorize and compare the results

list_clusters <- seq(5,15,5)
list_distrib <-  c("heterogeneous","equal")

# For a given number of clusters
for(num.clusters in list_clusters) {
    # Data distribution among clusters will either be heterogeneous or equal 
    for (size in list_distrib) {
        
        print("##########")
        print(paste0("-> Distribution: ", size, ", Nb clusters: ", num.clusters))
        
        # Make simulated data
        print("-> Simulating data...")
        simulated_data_generation(simul_folder, num.clusters, size, predefined=TRUE)
        # Run factorisation
        print("-> Running factorisation...")
        out <- runfactorization(simul_folder, paste0("omics",1:3,".txt"), num.clusters, sep="\t", filtering="none")
        # Folder for comparison 
        comp_folder <- paste0(results_folder, num.clusters, "_", size)
        dir.create(comp_folder)
        # Perform clustering and comparison of factorisations
        print("-> Comparing results...")
        clusters_comparison(out$factorizations, out$method, 
                            simul_folder, comp_folder, 
                            out$icluster.clusters, out$intNMF.clusters, num.clusters)
        
        print("-> Done.")
    }
}

[1] "##########"
[1] "-> Distribution: heterogeneous, Nb clusters: 5"
[1] "-> Simulating data..."
[1] 0.30381405 0.15618647 0.34518373 0.13078438 0.06403138
[1] 1
[1] TRUE
[1] "-> Running factorisation..."


Warning message in createMOFAobject(omics):
“View names are not specified in the data, renaming them to: view_1, view_2, view_3
”Creating MOFA object from list of matrices,
 please make sure that samples are columns and features are rows...

Checking data options...
Checking training options...
Checking model options...


[1] "No output file provided, using a temporary file..."
Generating warm start... 
K=6:12
[1] "-> Comparing results..."
[1] "-> Done."
[1] "##########"
[1] "-> Distribution: equal, Nb clusters: 5"
[1] "-> Simulating data..."
[1] 0.21 0.20 0.20 0.20 0.19
[1] 1
[1] TRUE
[1] "-> Running factorisation..."


Warning message in createMOFAobject(omics):
“View names are not specified in the data, renaming them to: view_1, view_2, view_3
”Creating MOFA object from list of matrices,
 please make sure that samples are columns and features are rows...

Checking data options...
Checking training options...
Checking model options...


[1] "No output file provided, using a temporary file..."
Generating warm start... 
K=6:12
[1] "-> Comparing results..."
[1] "-> Done."


## Generating results and plots

We here save to the Results folder the obtained Jaccard Indices and plot their values according to Figure 2 in the paper. The results that you will obtain will be slightly different from those of the paper, due to the stochasticity of the InterSIM package. The data that you will simulate will not have the same sample-cluster associations and the same features of those simulated in the paper. However, the jDRs having best performances remain the same despite such variations. For the clustering methods sometimes small variability in intNMF results is observed.   

In [8]:
# Save all boxplots in a single PDF output file
pdf(file=paste0(results_folder, "simulated_boxplots.pdf"), width = 15, height = 15, onefile = TRUE)

# For each chosen number of clusters
for (i in list_clusters) {
    
    # Output files for each distribution
    eq_file <- paste0(results_folder, i, "_equal/", "JI.txt")
    het_file <- paste0(results_folder, i, "_heterogeneous/", "JI.txt")
    if(exists("JI.final")) rm(JI.final)
    
    # Load clusters (equal distribution)
    if(file.exists(eq_file)) {
        JI.final  <- read.table(eq_file, sep="\t", header=TRUE)
        names(JI.final) <- paste0(names(JI.final), "_EQ")        
    }
    # Load clusters (heterogeneous distribution)
    if(file.exists(het_file)) {
        JI.het <- read.table(het_file, sep="\t", header=TRUE)
        names_methods <- names(JI.het)
        names(JI.het) <- paste0(names(JI.het), "_HET")

        # Aggregate results
        if(exists("JI.final")) {
            JI.final <- data.frame(JI.het, JI.final)
            new_order <- apply(expand.grid(c("_HET", "_EQ"), names_methods)[, c(2,1)], 1, paste, collapse="")
            JI.final <- JI.final[, new_order]
        }
        else {
            JI.final <- JI.het
        }
    }
    
    # Plot results
    par(mar=c(25,4,2,2)+.1)
    boxplot(JI.final, xaxt="none", cex.axis=3.5, 
                 col=c('gray','gray','red','red','blue','blue','blueviolet','blueviolet','deeppink','deeppink','chocolate1','chocolate1','darkgoldenrod','darkgoldenrod','green','green','darkturquoise','darkturquoise'), 
                 ann=FALSE, outline=FALSE)
    matplot(1:ncol(JI.final), t(JI.final), col="black", pch=16, xaxt="none", cex=0.8, add=TRUE)
    axis(1, at=1:ncol(JI.final), labels=names(JI.final), 
         las=2, srt=45, cex=0.8, cex.lab=3.5, cex.axis=3.5, cex.main=1.5, cex.sub=1.5) 
    title(main=paste(i,"clusters",sep=" "), 
          cex.lab=0.75, line = -2.5, adj=0, cex.main=3.5)
}
dev.off()

png 
  2